In [0]:
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libgle3
The following NEW packages will be installed:
  python-opengl
0 upgraded, 1 newly installed, 0 to remove and 25 not upgraded.
Need to get 496 kB of archives.
After this operation, 5,416 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-opengl all 3.1.0+dfsg-1 [496 kB]
Fetched 496 kB in 1s (496 kB/s)
Selecting previously unselected package python-opengl.
(Reading database ... 134448 files and directories currently installed.)
Preparing to unpack .../python-opengl_3.1.0+dfsg-1_all.deb ...
Unpacking python-opengl (3.1.0+dfsg-1) ...
Setting up python-opengl (3.1.0+dfsg-1) ...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 25 not upgraded.
Need to get 784 kB of 

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
!pip install ptan

     |████████████████████████████████| 773.1MB 21kB/s 
  Created wheel for ptan: filename=ptan-0.6-cp36-none-any.whl size=23502 sha256=b6059c446df6e98f63394e0d40f0a95b8120eb5c6aab2002ccd3679ed2335917
  Stored in directory: /root/.cache/pip/wheels/f0/4b/2f/9a45fd39b0a614a2716bc6128a7f1adb4647f323a2d90783f2
Successfully built ptan
ERROR: torchvision 0.5.0 has requirement torch==1.4.0, but you'll have torch 1.3.0 which is incompatible.
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0


In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

In [0]:
import torch
import torch.nn as n
import torch.nn.functional as f
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as op
import gym
import ptan

In [0]:
env=gym.make("CartPole-v0")

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [0]:
nactions=env.action_space.n

In [0]:
obssize=env.observation_space.shape[0]

In [0]:
class net(n.Module):
  def __init__(self,indim,middim,nactions):
    super().__init__()
    self.fc1=n.Linear(indim,middim)
    self.fc2=n.Linear(middim,nactions)

  def forward(self,x):
    out=f.relu(self.fc1(x))
    return self.fc2(out)

In [0]:
gamma=0.9

In [0]:
def calc_qvals(rew):
  qval=[]
  startrew=0
  for i in reversed(rew):
    startrew*=gamma
    startrew+=i
    qval.append(startrew)
  return list(reversed(qval))

In [0]:
net1=net(obssize,128,nactions)

In [0]:
agent1=ptan.agent.PolicyAgent(net1,preprocessor=ptan.agent.float32_preprocessor,apply_softmax=True)

In [0]:
expsource=ptan.experience.ExperienceSourceFirstLast(env,agent1,gamma=0.0,steps_count=1)

In [0]:
opt=op.Adam(net1.parameters(),lr=0.0001)

In [0]:
t=0
for i in expsource:
  print(i)
  t+=1
  if t==5:
    break

ExperienceFirstLast(state=array([-0.00492767,  0.0449467 , -0.01098882,  0.04703487]), action=0, reward=1.0, last_state=array([-0.00402874, -0.15001597, -0.01004813,  0.33623056]))
ExperienceFirstLast(state=array([-0.00402874, -0.15001597, -0.01004813,  0.33623056]), action=0, reward=1.0, last_state=array([-0.00702906, -0.34499349, -0.00332352,  0.62572799]))
ExperienceFirstLast(state=array([-0.00702906, -0.34499349, -0.00332352,  0.62572799]), action=0, reward=1.0, last_state=array([-0.01392893, -0.54006889,  0.00919104,  0.91736236]))
ExperienceFirstLast(state=array([-0.01392893, -0.54006889,  0.00919104,  0.91736236]), action=0, reward=1.0, last_state=array([-0.0247303 , -0.7353139 ,  0.02753829,  1.21291962]))
ExperienceFirstLast(state=array([-0.0247303 , -0.7353139 ,  0.02753829,  1.21291962]), action=0, reward=1.0, last_state=array([-0.03943658, -0.93078022,  0.05179668,  1.51410318]))


In [0]:
totalrewards=[]
doneepisodes=0
batchepisodes=[]
batchreward=0
batchqval=[]
batchactions=[]
batchstates=[]
currewards=[]

In [0]:
t=0
for step,exp in enumerate(expsource):
  batchstates.append(exp.state)
  batchactions.append(exp.action)
  currewards.append(exp.reward)
  if exp.last_state is None:
    batchqval.extend(calc_qvals(currewards))
    currewards.clear()
    batchreward+=1   #one episode done
  
  new_rewards=expsource.pop_total_rewards()

  if new_rewards:
    t+=1
    if t==5:
      break
      
    doneepisodes+=1
    reward=new_rewards[0]
    totalrewards.append(reward)
    meanrewards=float(np.mean(totalrewards[-100:]))
    print(meanrewards,step)
    if meanrewards>195:
      break

21.42 10
21.44 36
21.38 45
21.47 63


In [0]:
t=0
for i in expsource:
  print(i)
  t+=1
  if t==20:
    break

ExperienceFirstLast(state=array([-0.023885  ,  0.02987247,  0.04482912, -0.03882423]), action=1, reward=1.0, last_state=array([-0.02328755,  0.22432386,  0.04405263, -0.31703293]))
ExperienceFirstLast(state=array([-0.02328755,  0.22432386,  0.04405263, -0.31703293]), action=1, reward=1.0, last_state=array([-0.01880108,  0.41879157,  0.03771197, -0.59550422]))
ExperienceFirstLast(state=array([-0.01880108,  0.41879157,  0.03771197, -0.59550422]), action=0, reward=1.0, last_state=array([-0.01042525,  0.22316268,  0.02580189, -0.29118483]))
ExperienceFirstLast(state=array([-0.01042525,  0.22316268,  0.02580189, -0.29118483]), action=0, reward=1.0, last_state=array([-0.00596199,  0.02768251,  0.01997819,  0.00952274]))
ExperienceFirstLast(state=array([-0.00596199,  0.02768251,  0.01997819,  0.00952274]), action=0, reward=1.0, last_state=array([-0.00540834, -0.16772018,  0.02016865,  0.30844148]))
ExperienceFirstLast(state=array([-0.00540834, -0.16772018,  0.02016865,  0.30844148]), action=0

In [0]:
expsource.total_rewards

[]

In [0]:
expsource.pop_total_rewards()

[]